# Ejercicio: Fragmentador de capítulos en un documento

Define extensiones personalizadas para los objetos `Span` y `Doc`:

- Cada `Span` (capítulo) debe tener:
  - Su número (`numero`)
  - Su título (`titulo`)
  - Su longitud en tokens (`longitud`)

- El objeto `Doc` debe tener:
  - Una lista de todos los capítulos (`capitulos`)



In [25]:
import re
with open("libro1.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
import spacy
nlp = spacy.load("es_core_news_md")
doc = nlp(texto)
#importa la clase Span
from spacy.tokens import Span, Doc
#hagamos un span para cada capítulo. debe tener su número, su título y su longitud
if not Span.has_extension("numero"):
    # Registrar atributos personalizados en Span
    Span.set_extension("numero", default=None)
    '''Span.set_extension("titulo", default=None)
    Span.set_extension("longitud", default=None)
# Registrar atributos personalizados en Span
Span.set_extension("numero", default=None)'''
if not Span.has_extension("titulo"):
    # Registrar atributos personalizados en Span
    Span.set_extension("titulo", default=None)
if not Span.has_extension("longitud"):
    # Registrar atributos personalizados en Span
    Span.set_extension("longitud", default=None)
#Buscar encabezados de capítulo con regex: número al principio de línea seguido de texto
# Ejemplo: "1 El niño que vivió"
capitulos = list(re.finditer(r"(?m)^(\d+)\s+([A-Z][^\n]*)", texto))

# Lista para guardar la info de cada capítulo
capitulos_info = []

# Iterar por capítulos para construir spans
for i, match in enumerate(capitulos):
    numero = int(match.group(1))
    titulo = match.group(2).strip()

    # Índices del capítulo actual en caracteres
    start_char = match.end()

    # Determinar el final del capítulo: inicio del siguiente o final del texto
    if i + 1 < len(capitulos):
        end_char = capitulos[i + 1].start()
    else:
        end_char = len(texto)

    # Crear el span usando char_span
    span = doc.char_span(start_char, end_char, alignment_mode="expand")

    if span:
        # Atributos personalizados
        span._.numero = numero
        span._.titulo = titulo
        span._.longitud = len(span)

        capitulos_info.append(span)
    
resumen = "\n".join([
    f"{span._.numero} {span._.titulo} {span._.longitud}"
    for span in capitulos_info
])
resumen


# Crear un nuevo Doc con ese texto
doc_resumen = nlp.make_doc(resumen)

Crea un componente personalizado para añadir luego al pipeline, llamado `fragmentador_capitulos` que:

- Reciba un `Doc`
- Busque los capítulos utilizando una expresión regular, donde cada capítulo esté definido por un número seguido de su título.
- Cree un `Span` para cada capítulo con los atributos personalizados definidos.
- Guarde la lista de capítulos en el atributo `doc._.capitulos`.

In [26]:
from spacy.language import Language
@Language.component("fragmentador_capitulos")
def fragmentador_capitulos(doc):
    texto = doc.text

    # Buscar encabezados de capítulos: número + título
    capitulos = list(re.finditer(r"(?m)^(\d+)\s+([A-Z][^\n]*)", texto))
    spans = []

    for i, match in enumerate(capitulos):
        numero = int(match.group(1))
        titulo = match.group(2).strip()
        start_char = match.end()
        end_char = capitulos[i + 1].start() if i + 1 < len(capitulos) else len(texto)

        # Crear Span desde el texto original
        span = doc.char_span(start_char, end_char, alignment_mode="expand")

        if span:
            span._.numero = numero
            span._.titulo = titulo
            span._.longitud = len(span)
            spans.append(span)

    # Guardar la lista en el atributo del Doc
    doc._.capitulos = spans

    return doc
nlp.add_pipe("fragmentador_capitulos", last=True)


<function __main__.fragmentador_capitulos(doc)>

# Ejercicio: Segmentación en capítulos del tercer libro de Harry Potter

- Define la variable `nlp` con el modelo `es_core_news_md`.
- Añade el componente que has definirdo al pipeline de spaCy. Respecto a este pipeline, en el resto de esta notebook no será necesario utilizar el NER predefinido, ya que nuestro interés será detectar otro tipo de entidades que nosotros mismos vamos a definir. 
- Carga el tercer libro de Harry Potter a Partir de la línea 37. 
- Procesa el texto del tercer libro de Harry Potter con `nlp`, y comprueba que el `Doc` resultante contenga todos los capítulos segmentados y anotados correctamente.

In [27]:
if not spacy.tokens.Doc.has_extension("capitulos"):
    spacy.tokens.Doc.set_extension("capitulos", default=[])
nlp = spacy.load("es_core_news_md")
nlp.add_pipe("fragmentador_capitulos", last=True)
with open("libro3.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
doc = nlp(texto)


In [28]:
for capitulo in doc._.capitulos:
    print(f"Capítulo {capitulo._.numero}: {capitulo._.titulo} (Longitud: {capitulo._.longitud} caracteres)")

Capítulo 1: Lechuzas mensajeras (Longitud: 4418 caracteres)
Capítulo 2: El error de tía Marge (Longitud: 4651 caracteres)
Capítulo 3: El autobús noctámbulo (Longitud: 5433 caracteres)
Capítulo 4: El Caldero Chorreante (Longitud: 6465 caracteres)
Capítulo 5: El dementor (Longitud: 8178 caracteres)
Capítulo 6: Posos de té y garras de hipogrifo (Longitud: 8244 caracteres)
Capítulo 7: El boggart del armario ropero (Longitud: 5464 caracteres)
Capítulo 8: La huida de la señora gorda (Longitud: 6202 caracteres)
Capítulo 9: La derrota (Longitud: 6388 caracteres)
Capítulo 10: El mapa del merodeador (Longitud: 8671 caracteres)
Capítulo 11: La Saeta de Fuego (Longitud: 6762 caracteres)
Capítulo 12: El patronus (Longitud: 5809 caracteres)
Capítulo 13: Gryffindor contra Ravenclaw (Longitud: 5152 caracteres)
Capítulo 14: El rencor de Snape (Longitud: 6651 caracteres)
Capítulo 15: La final de quidditch (Longitud: 6964 caracteres)
Capítulo 16: La predicción de la profesora Trelawney (Longitud: 5275 ca

## Ejercicio: Detectar hechizos y añadirlos como entidades

Utilizando el objeto `PhraseMatcher` de spaCy, crea un sistema que detecte menciones de hechizos en un texto y los añada como nuevas entidades. Utiliza la siguiente lista de hechizos:

`hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]`

Realiza este ejercicio de dos formas diferentes y razona qué está ocurriendo. 



In [29]:
import spacy
from spacy.matcher import PhraseMatcher

# Lista de hechizos
hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]

### Debemos desactivar el name entity recognizer para evitar conflictos con el PhraseMatcher
nlp.disable_pipes("ner")


# Crear el PhraseMatcher y agregar las frases correspondientes
matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp.make_doc(hechizo) for hechizo in hechizos]
matcher.add("HECHIZOS", None, *patterns)

# Función para añadir los hechizos como nuevas entidades
@Language.component("detectar_hechizos")
def detectar_hechizos(doc):
    doc.ents = []
    matches = matcher(doc)
    # Crear entidades en el doc para cada coincidencia
    for match_id, start, end in matches:
        span = Span(doc, start, end, label="HECHIZO")
        doc.ents = list(doc.ents) + [span]  # Añadir la nueva entidad al doc
    return doc

# Agregar el componente a la pipeline
nlp.add_pipe('detectar_hechizos', last=True)

<function __main__.detectar_hechizos(doc)>

1. Con el `texto` del tercer libro cargado tal cual

In [30]:
with open("libro3.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
doc = nlp(texto)

# Ver las entidades detectadas
for ent in doc.ents:
    if ent.label_ == "HECHIZO":  # Filtrar solo las entidades de tipo 'HECHIZO'
        print(f"Entidad: {ent.text}, Etiqueta: {ent.label_}")

Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO


2. Haciendo la modificación `texto = texto.replace("", " ").replace("\x97", " ")` después de cargar el tercer libro.

In [31]:
with open("libro3.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
texto = texto.replace("", " ").replace("\x97", " ")
doc = nlp(texto)

# Ver las entidades detectadas
for ent in doc.ents:
    if ent.label_ == "HECHIZO":  # Filtrar solo las entidades de tipo 'HECHIZO'
        print(f"Entidad: {ent.text}, Etiqueta: {ent.label_}")

Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Expeliarmo, Etiqueta: HECHIZO
Entidad: Expeliarmo, Etiqueta: HECHIZO
Entidad: Expeliarmo, Etiqueta: HECHIZO
Entidad: Expeliarmo, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etique

## Acceder al texto entre los títulos de los capítulos


In [32]:
for i, capitulo in enumerate(doc._.capitulos):
    # Obtener el texto del capítulo
    texto_capitulo = capitulo.text

    # Obtener el índice del último carácter del título dentro del capítulo
    indice_titulo = texto_capitulo.rfind(capitulo._.titulo) + len(capitulo._.titulo)

    # Convertir a índice absoluto dentro del texto completo
    indice_titulo_absoluto = capitulo.start_char + indice_titulo

    # Obtener el índice del primer carácter del siguiente capítulo
    if i + 1 < len(doc._.capitulos):
        siguiente_capitulo = doc._.capitulos[i + 1]
        indice_siguiente = siguiente_capitulo.start_char
    else:
        indice_siguiente = len(doc.text)

    # Extraer el texto entre esos índices
    texto_intermedio = doc.text[indice_titulo_absoluto:indice_siguiente].strip()

    # Imprimir el resultado
    print(f"Texto entre capítulos {capitulo._.numero} y {capitulo._.numero + 1}:")
    print(texto_intermedio)


Texto entre capítulos 1 y 2:
a, en muchos sentidos, un muchacho diferente. Por un lado, las vacaciones de verano le gustaban menos que cualquier otra época del año; y por otro, deseaba de verdad hacer los deberes, pero tenía que hacerlos a escondidas, muy entrada la noche. Y además, Harry Potter era un mago.
Era casi medianoche y estaba tumbado en la cama, boca abajo, tapado con las mantas hasta la cabeza, como en una tienda de campaña. En una mano tenía la linterna y, abierto sobre la almohada, había un libro grande, encuadernado en piel (Historia de la Magia, de Adalbert Waffling). Harry recorría la página con la punta de su pluma de águila, con el entrecejo fruncido, buscando algo que le sirviera para su redacción sobre «La inutilidad de la quema de brujas en el siglo XIV».
La pluma se detuvo en la parte superior de un párrafo que podía serle útil. Harry se subió las gafas redondas, acercó la linterna al libro y leyó:

En la Edad Media, los no magos (comúnmente denominados muggles) 

## Seleccionamos las frases que contienen hechizos

In [33]:
hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]
hechizos_lower = [h.lower() for h in hechizos]
texto = texto.replace("", " ").replace("\x97", " ")
# Dividimos el texto en frases usando signos de puntuación
frases = re.split(r'(?<=[\.\?!])\s+', texto)

# Seleccionamos frases que contengan algún hechizo
frases_con_hechizos = []
for frase in frases:
    frase_lower = frase.lower()
    for hechizo in hechizos_lower:
        if hechizo in frase_lower:
            frases_con_hechizos.append(frase)
            break

# Mostramos algunas frases encontradas
for frase in frases_con_hechizos[:10]:  # Muestra las primeras 10
    print(frase)
print(f"Total de frases encontradas: {len(frases_con_hechizos)}")


¡Lumos!
Levantó la varita, murmuró ¡Lumos!, y vio que se encontraba en un pasadizo muy estrecho, bajo y cubierto de barro.
El encantamiento es así  Lupin se aclaró la garganta : ¡Expecto patronum!
¡Expecto patronum!
¡Expecto patronum!
¡Expecto patronum!
¡Expecto patronum!
¡Expecto patronum!
¡Expecto patronum!
¡Expecto patronum!
Total de frases encontradas: 34


## Conjunto de entrenamiento


In [34]:
TRAIN_DATA = []

for frase in frases_con_hechizos:
    entidades = []
    for hechizo in hechizos:
        # Buscar todas las apariciones del hechizo en la frase (may/min insensible)
        for match in re.finditer(re.escape(hechizo), frase, re.IGNORECASE):
            start, end = match.start(), match.end()
            entidades.append((start, end, "HECHIZO"))

    if entidades:
        TRAIN_DATA.append((frase, {"entities": entidades}))

# Mostrar algunos ejemplos
for ejemplo in TRAIN_DATA[:5]:
    print(ejemplo)

print(f"Total de ejemplos en TRAIN_DATA: {len(TRAIN_DATA)}")

('¡Lumos!', {'entities': [(1, 6, 'HECHIZO')]})
('Levantó la varita, murmuró ¡Lumos!, y vio que se encontraba en un pasadizo muy estrecho, bajo y cubierto de barro.', {'entities': [(28, 33, 'HECHIZO')]})
('El encantamiento es así  Lupin se aclaró la garganta : ¡Expecto patronum!', {'entities': [(56, 72, 'HECHIZO')]})
('¡Expecto patronum!', {'entities': [(1, 17, 'HECHIZO')]})
('¡Expecto patronum!', {'entities': [(1, 17, 'HECHIZO')]})
Total de ejemplos en TRAIN_DATA: 34


## Entrenar el modelo NER

In [35]:
import spacy
from spacy.training.example import Example

# Crear un modelo en blanco para español
nlp = spacy.blank("es")

# Añadir el componente de NER
ner = nlp.add_pipe("ner")

# Añadir las etiquetas de entidad
ner.add_label("HECHIZO")

# Comenzar entrenamiento
nlp.begin_training()

# Entrenamiento en varias iteraciones
for epoch in range(30):  # puedes ajustar el número de epochs
    losses = {}
    for text, annotations in TRAIN_DATA:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], losses=losses)
    print(f"Epoch {epoch+1}, Losses: {losses}")

# Guardar el modelo entrenado
nlp.to_disk("modelo_hechizos")
print("✅ Modelo entrenado y guardado en 'modelo_hechizos'")


Epoch 1, Losses: {'ner': np.float32(70.87463)}
Epoch 2, Losses: {'ner': np.float32(9.969383)}
Epoch 3, Losses: {'ner': np.float32(0.07083751)}
Epoch 4, Losses: {'ner': np.float32(3.069415e-06)}
Epoch 5, Losses: {'ner': np.float32(4.839299e-07)}
Epoch 6, Losses: {'ner': np.float32(3.240109e-06)}
Epoch 7, Losses: {'ner': np.float32(1.4409704e-07)}
Epoch 8, Losses: {'ner': np.float32(3.5341236e-07)}
Epoch 9, Losses: {'ner': np.float32(9.955243e-08)}
Epoch 10, Losses: {'ner': np.float32(5.656674e-07)}
Epoch 11, Losses: {'ner': np.float32(7.663006e-08)}
Epoch 12, Losses: {'ner': np.float32(5.3033027e-08)}
Epoch 13, Losses: {'ner': np.float32(6.873607e-08)}
Epoch 14, Losses: {'ner': np.float32(1.46240104e-08)}
Epoch 15, Losses: {'ner': np.float32(2.3116264e-08)}
Epoch 16, Losses: {'ner': np.float32(2.4141285e-08)}
Epoch 17, Losses: {'ner': np.float32(2.3842276e-07)}
Epoch 18, Losses: {'ner': np.float32(8.3154596e-08)}
Epoch 19, Losses: {'ner': np.float32(4.5591526e-09)}
Epoch 20, Losses: {'n

## Usar el modelo

In [36]:
from textwrap import wrap

# Cargar modelo
nlp = spacy.load("modelo_hechizos")

# Leer texto a partir de la línea 38
with open("libro4.txt", "r", encoding="utf-8") as archivo:
    texto4 = "".join(archivo.readlines()[37:])

# Dividir el texto en fragmentos (ajusta el tamaño según tu RAM)
fragmentos = wrap(texto4, 5000)

# Guardar resultados
resultados = []

for i, fragmento in enumerate(fragmentos):
    doc = nlp(fragmento)
    for ent in doc.ents:
        if ent.label_ == "HECHIZO":
            resultados.append({
                "fragmento": i + 1,
                "hechizo": ent.text,
                "start": ent.start_char,
                "end": ent.end_char
            })

# Mostrar algunos resultados
for r in resultados[:10]:
    print(f"[Fragmento {r['fragmento']}] {r['hechizo']} (pos: {r['start']}–{r['end']})")

print(f"\nTotal de hechizos detectados: {len(resultados)}")



[Fragmento 1] Están (pos: 1385–1390)
[Fragmento 1] Y (pos: 1413–1414)
[Fragmento 1] Siempre (pos: 3349–3356)
[Fragmento 1] Y (pos: 3446–3447)
[Fragmento 1] Entonces (pos: 4621–4629)
[Fragmento 3] Magia (pos: 2500–2505)
[Fragmento 3] Estarán (pos: 2632–2639)
[Fragmento 3] Magia (pos: 2885–2890)
[Fragmento 3] Evidentemente (pos: 2893–2906)
[Fragmento 3] Quién (pos: 4471–4476)

Total de hechizos detectados: 1289


## Regex para los libros 4 y 5

### libro 4

In [39]:
import re
import pandas as pd
from spacy.tokens import Span
if not Span.has_extension("numero"):
    # Registrar atributos personalizados en Span
    Span.set_extension("numero", default=None)
    '''Span.set_extension("titulo", default=None)
    Span.set_extension("longitud", default=None)
# Registrar atributos personalizados en Span
Span.set_extension("numero", default=None)'''
if not Span.has_extension("titulo"):
    # Registrar atributos personalizados en Span
    Span.set_extension("titulo", default=None)
if not Span.has_extension("longitud"):
    # Registrar atributos personalizados en Span
    Span.set_extension("longitud", default=None)

with open("libro4.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
nlp = spacy.load("es_core_news_md")
nlp.max_length = 1200000 
doc = nlp(texto)
#Buscar encabezados de capítulo con regex: número al principio de línea seguido de texto
# Ejemplo: "1 El niño que vivió"
capitulos = list(re.finditer(r"(?m)^\s*(\d+)\s*\n([^\n]+)", texto))





# Lista para guardar la info de cada capítulo
capitulos_info = []

# Iterar por capítulos para construir spans
for i, match in enumerate(capitulos):
    numero = int(match.group(1))
    titulo = match.group(2).strip()

    # Índices del capítulo actual en caracteres
    start_char = match.end()

    # Determinar el final del capítulo: inicio del siguiente o final del texto
    if i + 1 < len(capitulos):
        end_char = capitulos[i + 1].start()
    else:
        end_char = len(texto)

    # Crear el span usando char_span
    span = doc.char_span(start_char, end_char, alignment_mode="expand")

    if span:
        # Atributos personalizados
        span._.numero = numero
        span._.titulo = titulo
        span._.longitud = len(span)

        capitulos_info.append({
            "numero": numero,
            "titulo": titulo,
            "longitud": len(span),
        })

# Crear DataFrame
df = pd.DataFrame(capitulos_info)

# Mostrar resultado
print(df)

    numero                                           titulo  longitud
0        1                         La Mansión de los Ryddle      5407
1        2                                      La cicatriz      3543
2        3                                    La invitación      4007
3        4                          Retorno a La Madriguera      3923
4        5                              Sortilegios Weasley      4849
5        6                                     El traslador      3205
6        7                                  Bagman y Crouch      7015
7        8                       Los Mundiales de quidditch      8154
8        9                               La Marca Tenebrosa      9695
9       10                        Alboroto en el Ministerio      4285
10      11                        En el expreso de Hogwarts      4453
11      12                      El Torneo de los tres magos      7543
12      13                                    Ojoloco Moody      5366
13      14          

### libro 5

In [53]:
import re
import pandas as pd
import spacy
from spacy.tokens import Span

# Función para convertir números romanos a decimales
def romano_a_decimal(romano):
    romanos = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
    decimal = 0
    prev_value = 0
    for letra in reversed(romano):
        value = romanos[letra]
        if value < prev_value:
            decimal -= value
        else:
            decimal += value
        prev_value = value
    return decimal

# Verificar y registrar extensiones personalizadas en Span
if not Span.has_extension("numero"):
    Span.set_extension("numero", default=None)
if not Span.has_extension("titulo"):
    Span.set_extension("titulo", default=None)
if not Span.has_extension("longitud"):
    Span.set_extension("longitud", default=None)

# Cargar el archivo de texto
with open("libro5.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)

# Cargar el modelo de spacy en español
nlp = spacy.load("es_core_news_md")
nlp.max_length = 1600000
doc = nlp(texto)

# Buscar encabezados de capítulo con regex: número romano al principio de línea seguido de texto
# Ejemplo: "CAPÍTULO I El niño que vivió"
capitulos = list(re.finditer(r"(?m)^CAP[IÍ]TULO\s+([IVXLCDM]+)\s*\n+\s*([A-ZÁÉÍÓÚÑÜ][^\n]*)", texto))


# Lista para guardar la información de cada capítulo
capitulos_info = []

# Iterar por capítulos para construir spans
for i, match in enumerate(capitulos):
    romano = match.group(1)
    numero = romano_a_decimal(romano)  # Convertir número romano a decimal
    titulo = match.group(2).strip()

    # Índices del capítulo actual en caracteres
    start_char = match.start()

    # Determinar el final del capítulo: inicio del siguiente o final del texto
    if i + 1 < len(capitulos):
        end_char = capitulos[i + 1].start()
    else:
        end_char = len(texto)

    # Crear el span usando char_span
    span = doc.char_span(start_char, end_char, alignment_mode="expand")

    if span:
        # Atributos personalizados
        span._.numero = numero
        span._.titulo = titulo
        span._.longitud = len(span)

        capitulos_info.append({
            "numero": numero,
            "titulo": titulo,
            "longitud": len(span),
        })

# Crear DataFrame
df = pd.DataFrame(capitulos_info)

# Mostrar resultado
print(df)



   numero                        titulo  longitud
0      34  EL DEPARTAMENTO DE MISTERIOS     39299
